## Работа с папками и файлами с помощью библиотеки OS  

Пусть имеются данные со следующей структурой:

- записываются для каждого пользователя, совершившего покупки, каждый день
- для каждой даты есть своя папка, внутри неё – папки для каждого пользователя
- внутри каждой папки есть файл data.csv, где и хранятся данные  
  
Схематично выглядит так:  
  
 └── data_os  
. . . ├── 2020-12-03   
. . . │ . ├── FirstName_LastName1   
. . . │ . │  . └── data.csv  
. . . │ . ├── FirstName_LastName2   
. . . │ . │  . └── data.csv  
. . . │ . └── FirstName_LastName3   
. . . │ . . . . .└── data.csv   
. . . └── 2020-12-04   
. . . . .. .├── FirstName_LastName1      
. . . . .. .│ . . └── data.csv      
. . . . . ..└── FirstName_LastName5      
. . . . . . . .  . └── data.csv  
  
Например, 03 декабря три покупателя сделали покупки, 04 – два (папки 2020-12-03 и 2020-12-04 соответственно). Поскольку клиент FirstName_LastName1 купил товары в оба дня, для него имеется папка в папке для каждой из дат. Для других клиентов – по одной.

  
**Задача**  
Собрать все данные из папки data_os в один датафрэйм, имеющий следующие столбцы: колонки из самих файлов (product_id, quantity), а также имя пользователя (name), и дата этих покупок (date), соответствует названию папки, где лежит папка с пользователем



**Применяемые методы**  
- Библиотеки

In [1]:
import pandas as pd
import os

  
  
- Метод  **os.listdir()** принимает путь к папке и возвращает её содержимое в виде списка

In [2]:
root_path = 'DATA/data_os'
os.listdir(root_path)      # список папок

['2020-12-03',
 '2020-12-04',
 '2020-12-05',
 '2020-12-06',
 '2020-12-07',
 '2020-12-08',
 '2020-12-09']

- При вложенности папок и необходимости добраться до дна можно использовать  **os.walk()**.  
os.walk() возвращает три значения: путь к папке, список папок, список файлов на каждой итерации   
DATA/data_os,  ['2020-12-03',  '2020-12-04',  '2020-12-05',  ...], [ ]  
. . . . . . . . . . . . .  DATA/data_os/2020-12-03,  ['Alexey_Smirnov',  'Anton_Petrov',  ... ], [ ]  
. . . .  .  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . DATA/data_os/2020-12-03/Alexey_Smirnov,  [ ],  ['data.csv']  
  
- Чтобы получить полный адрес файла, следует воспользоваться функцией **os.path.join**.  
Функция принимает путь и компонент(ы) пути (например, имя файла).

In [3]:
for path, dirs, files in os.walk(root_path):
    for name in files:
        print(os.path.join(path, name))

DATA/data_os\2020-12-03\Alexey_Smirnov\data.csv
DATA/data_os\2020-12-03\Anton_Petrov\data.csv
DATA/data_os\2020-12-03\Kirill_Petrov\data.csv
DATA/data_os\2020-12-03\Petr_Fedorov\data.csv
DATA/data_os\2020-12-03\Vasiliy_Ivanov\data.csv
DATA/data_os\2020-12-03\Vasiliy_Petrov\data.csv
DATA/data_os\2020-12-04\Alexey_Smirnov\data.csv
DATA/data_os\2020-12-04\Kirill_Fedorov\data.csv
DATA/data_os\2020-12-04\Petr_Fedorov\data.csv
DATA/data_os\2020-12-04\Petr_Ivanov\data.csv
DATA/data_os\2020-12-04\Petr_Smirnov\data.csv
DATA/data_os\2020-12-04\Rostislav_Fedorov\data.csv
DATA/data_os\2020-12-04\Rostislav_Ivanov\data.csv
DATA/data_os\2020-12-04\Rostislav_Smirnov\data.csv
DATA/data_os\2020-12-04\Vasiliy_Ivanov\data.csv
DATA/data_os\2020-12-05\Alexey_Fedorov\data.csv
DATA/data_os\2020-12-05\Alexey_Petrov\data.csv
DATA/data_os\2020-12-05\Kirill_Smirnov\data.csv
DATA/data_os\2020-12-05\Petr_Ivanov\data.csv
DATA/data_os\2020-12-05\Petr_Smirnov\data.csv
DATA/data_os\2020-12-05\Rostislav_Smirnov\data.csv

- **Функция concat_csv()**  
Создадим функцию concat_csv(root_path), которая принимает путь к папке в виде строки (root_path) и название файла в виде строки, считывает данные из csv-файлов во всех вложенных папках, считывает названия папок как строки и объединяет все считанные данные в один датафрейм.  

In [11]:
def concat_csv(root_path, file_name='data.csv'):
    
    """
    Данная функция принимает путь к папке и возвращает датафрейм пандаса.

    Parameters
    ---------
    root_path: str
        Путь к папке
    file_name: str, default 'data.csv'
        Название csv-файла в формате 'name.csv'
        Если этот параметр не указан, то считываются файлы с названием 'data.csv'

    Returns: DataFrame
    """
    
    # Создаем пустой датафрейм df1
    df1 = pd.DataFrame()
    
    # Итерация по списку папок в корневой папке
    for date in os.listdir(root_path):             # каждая папка - дата (date)
        user_path = os.path.join(root_path, date)  # сохраняем путь корневая папка + вложенная папка 1-го уровня
        
        # Итерация по списку папок во вложенной папке 1-го уровня
        for user in os.listdir(user_path):    # каждая папка - Имя_Фамилия (user)
            
            # Получаем путь каждого csv-файла с помощью os.path.join
            # Считываем каждый файл в датафрейм df2 с помощью read_csv
            df2 = pd.read_csv(os.path.join(user_path, user, file_name))
            
            # Добавляем в датафрейм df2 колонки даты (date) и имени покупателя (user) из переменных цикла (они же - названия папок)
            df2['date'], df2['user'] = date, user
            
            # Присоединяем считанный датафрейм df2 к созданному ранее df1 с помощью pd.concat (вертикально по умолчанию)
            df1 = pd.concat([df1, df2], ignore_index=True)    # индексы будут по порядку
    
    # Удаляем колонку со старыми индексами с помощью drop
    df1 = df1.drop(columns='Unnamed: 0')
    
    # Возвращается итоговый датафрейм
    return df1

**Результат работы функции**

In [10]:
# Вызов функции

concat_csv(root_path)

,product_id,quantity,date,user
0,47,1,2020-12-03,Alexey_Smirnov
1,57,1,2020-12-03,Alexey_Smirnov
2,14,2,2020-12-03,Alexey_Smirnov
3,52,1,2020-12-03,Alexey_Smirnov
4,1,4,2020-12-03,Alexey_Smirnov
...,...,...,...,...
156,66,4,2020-12-09,Rostislav_Petrov
157,39,2,2020-12-09,Rostislav_Petrov
158,82,4,2020-12-09,Vasiliy_Ivanov
159,74,2,2020-12-09,Vasiliy_Ivanov
